In [22]:
from google.colab import files
import pandas as pd

uploaded = files.upload()
filename = list(uploaded.keys())[0]
df = pd.read_csv(filename)

df.head()

Saving spam_assassin.csv to spam_assassin.csv


,text,target
0,From ilug-admin@linux.ie Mon Jul 29 11:28:02 2...,0
1,From gort44@excite.com Mon Jun 24 17:54:21 200...,1
2,From fork-admin@xent.com Mon Jul 29 11:39:57 2...,1
3,From dcm123@btamail.net.cn Mon Jun 24 17:49:23...,1
4,From ilug-admin@linux.ie Mon Aug 19 11:02:47 2...,0


In [23]:
import re

def extract_body(text):
    if pd.isna(text):
        return ""

    # List of all header fields found in SpamAssassin dumps
    header_keywords = [
        "From:", "To:", "Subject:", "Date:", "Return-Path:", "Delivered-To:",
        "Received:", "Message-Id:", "Content-Type:", "Content-Disposition:",
        "User-Agent:", "X-", "List-", "Mail-Followup-To:", "References:",
        "In-Reply-To:", "Errors-To:", "Sender:"
    ]

    last_index = 0

    # Find last header position in single-line email
    for key in header_keywords:
        matches = list(re.finditer(key, text, flags=re.IGNORECASE))
        if matches:
            last_index = max(last_index, matches[-1].end())

    body = text[last_index:].strip()
    return body


In [24]:
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def clean_text(text):
    if pd.isna(text):
        return ""

    text = text.lower()

    text = re.sub(r"http\S+|www\S+|https\S+", " ", text)  # URLs
    text = re.sub(r"\S+@\S+", " ", text)                  # emails
    text = re.sub(r"<.*?>", " ", text)                   # HTML tags
    text = re.sub(r"\d+", " ", text)                     # numbers
    text = text.translate(str.maketrans("", "", string.punctuation))  # punctuation
    text = re.sub(r"\s+", " ", text).strip()             # extra spaces

    # remove stopwords + lemmatization
    words = [
        lemmatizer.lemmatize(word)
        for word in text.split()
        if word not in stop_words
    ]

    return " ".join(words)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [25]:
df["body"] = df["text"].apply(extract_body)
df["clean_text"] = df["body"].apply(clean_text)

df = df[df["clean_text"].str.strip() != ""]  # remove empty rows

df.head()


,text,target,body,clean_text
0,From ilug-admin@linux.ie Mon Jul 29 11:28:02 2...,0,"Beenthere: ilug@linux.ie On Fri, Jul 26, 2002 ...",beenthere fri jul pm rumoured hereabouts wynne...
1,From gort44@excite.com Mon Jun 24 17:54:21 200...,1,Keywords: Content-Transfer-Encoding: 7bit Mort...,keywords contenttransferencoding bit mortgage ...
2,From fork-admin@xent.com Mon Jul 29 11:39:57 2...,1,"Fri, 26 Jul 0102 18:03:12 +0900 Content-Transf...",fri jul contenttransferencoding bit dear homeo...
4,From ilug-admin@linux.ie Mon Aug 19 11:02:47 2...,0,Beenthere: ilug@linux.ie Kevin O' Riordan's [k...,beenthere kevin riordans line wisdom included ...
5,From tobaccodemon@terra.es Sat Sep 7 22:05:58 ...,1,smokersclub@terra.es Yours faithfully. Smokers...,faithfully smoker united xay


In [26]:
output_name = "spam_cleaned_final.csv"
df.to_csv(output_name, index=False)

files.download(output_name)
print("Cleaning Done & File Downloading...")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Cleaning Done & File Downloading...
